### KANERVA2009

Hyperdimensional Computing: An Introduction to Computing in Distributed Representation with High-Dimensional Random Vectors

---

**Abstract.** The 1990s saw the emergence of cognitive models that depend on very high dimensionality and randomness. They include Holographic Reduced Representations, Spatter Code, Semantic Vectors, Latent Semantic Analysis, Context-Dependent Thinning, and Vector-Symbolic Architecture. They represent things in high-dimensional vectors that are manipulated by operations that produce new high-dimensional vectors in the style of traditional computing, in what is called here hyper-dimensional computing on account of the very high dimensionality. The paper presents the main ideas behind these models, written as a tutorial essay in hopes of making the ideas accessible and even provocative. A sketch of how we have arrived at these models, with references and pointers to further reading, is given at the end. The thesis of the paper is that hyper-dimensional representation has much to offer to students of cognitive science, theoretical neuroscience, computer science and engineering, and mathematics.

---


In [1]:
import numpy as np
import math
import random
from functools import reduce

dimensions = 10000

In [2]:
def generateRandomVector(d):
    return np.random.randint(2, size=d)

In [3]:
A = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

In [4]:
def similarity(A, B):
    if A.size != B.size:
        print("A and B have different dimensions.")

    count = 0
    for i in range(0, (A.size - 1)):
        if A[i] == B[i]:
            count += 1

    return count / A.size

In [5]:
def calculateSTDs(A, B):
    return abs((similarity(A, B) * A.size - 5000) / 50)

In [6]:
def applyThreshold(V, n):
    if n % 2 == 1:
        threshold = lambda x: 1 if x > n / 2 else 0
    else:
        threshold = (
            lambda x: 1
            if x > n / 2
            else (1 if x == n / 2 and bool(random.getrandbits(1)) else 0)
        )

    vectorized_threshold = np.vectorize(threshold)
    return vectorized_threshold(V)

In [7]:
def arithmeticSum(A, B):
    sum = A + B

    return applyThreshold(sum, 2)

In [8]:
def arithmeticSubtraction(A, B):
    compliment = 1 - B

    return arithmeticSum(A, compliment)

In [9]:
def arithmeticSumVectors(vectors):
    sum = reduce(lambda x, y: x + y, vectors)

    return applyThreshold(sum, len(vectors))

In [10]:
mean = arithmeticSum(A, B)

In [11]:
def comparison(A, B):
    dividend = sum(A * B)
    divisor = math.sqrt(sum(A**2) * sum(B**2))

    return dividend / divisor

In [12]:
print("sim  A, A:         ", similarity(A, A))
print("sim  A, B:         ", similarity(A, B))
print("sim  A, mean:      ", similarity(A, mean))
print("sim  B, mean:      ", similarity(B, mean))

print("comp A, B:         ", comparison(A, B))
print("comp A, mean:      ", comparison(A, mean))
print("comp B, mean:      ", comparison(B, mean))

print("STDs A, B:         ", calculateSTDs(A, B))
print("STDs A, mean:     ", calculateSTDs(A, mean))
print("STDs B, mean:     ", calculateSTDs(B, mean))

print("sim  A, mean - B:  ", similarity(A, arithmeticSubtraction(mean, B)))
print("STDs A, mean - B: ", calculateSTDs(A, arithmeticSubtraction(mean, B)))

print("sim  B, mean - B:  ", similarity(B, arithmeticSubtraction(mean, B)))
print("STDs B, mean - B: ", calculateSTDs(B, arithmeticSubtraction(mean, B)))

sim  A, A:          0.9999
sim  A, B:          0.5021
sim  A, mean:       0.7484
sim  B, mean:       0.7536
comp A, B:          0.507293303285248
comp A, mean:       0.7480506418534152
comp B, mean:       0.7555862063283526
STDs A, B:          0.42
STDs A, mean:      49.67999999999998
STDs B, mean:      50.72000000000002
sim  A, mean - B:   0.6235
STDs A, mean - B:  24.02
sim  B, mean - B:   0.3788
STDs B, mean - B:  24.88


In [13]:
def mult(A, B):
    return np.bitwise_xor(A, B)

In [14]:
def test_own_inverse_xor():
    A = generateRandomVector(dimensions)
    B = generateRandomVector(dimensions)

    assert comparison(A, mult(mult(A, B), B)) > 0.99

In [15]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanUp(self, V):
        return max(self.vectors, key=lambda x: comparison(V, x[1]))

In [16]:
itemMemory = ItemMemory()
itemMemory.addVector("A", A)
itemMemory.addVector("B", B)
itemMemory.addVector("mean", mean)

In [17]:
print(itemMemory.cleanUp(B)[0])

B


In [18]:
itemMemoryA = ItemMemory()

X = generateRandomVector(dimensions)
A = generateRandomVector(dimensions)

Y = generateRandomVector(dimensions)
B = generateRandomVector(dimensions)

Z = generateRandomVector(dimensions)
C = generateRandomVector(dimensions)

itemMemoryA.addVector("A", A)
itemMemoryA.addVector("B", B)
itemMemoryA.addVector("C", C)
itemMemoryA.addVector("X", X)
itemMemoryA.addVector("Y", Y)
itemMemoryA.addVector("Z", Z)

H = arithmeticSumVectors([mult(A, X), mult(B, Y), mult(C, Z)])

V = itemMemory.cleanUp(mult(H, X))

print(V[0])
print(similarity(mult(H, X), A))

A
0.7576


In [19]:
def generatePermutationMatrix(dimensions):
    P = np.eye(dimensions)
    np.random.shuffle(P)

    return P

In [20]:
def test_permutation_keeps_distance():
    N = generateRandomVector(dimensions)
    M = generateRandomVector(dimensions)

    nmMean = arithmeticSum(N, M)

    P = generatePermutationMatrix(dimensions)

    disB = similarity(N, nmMean)
    disA = similarity(P.dot(N), P.dot(nmMean))

    assert disB - disA < 0.01

In [21]:
def inversePermutation(P):
    return np.linalg.inv(P)

In [22]:
def test_inverse_permutation_cancels():
    V = generateRandomVector(dimensions)
    P = generatePermutationMatrix(dimensions)
    iP = inversePermutation(P)

    assert comparison(iP.dot(P.dot(V)), V) > 0.99
    assert similarity(iP.dot(P.dot(V)), V) > 0.99

In [23]:
test_permutation_keeps_distance()
test_inverse_permutation_cancels()
test_own_inverse_xor()